## ADES demonstration - OGC API Processes 
Deploy a CWL application package as a WPS processing service and submit an execution request

In [ ]:
import requests
import os
import getpass
import json
import yaml
from time import sleep

### Discover application published in the Resource Catalogue

In [ ]:
# Resource Catalogue endpoint
# ---------------------------
rm_endpoint = 'https://catalog.terradue.com/eoepca-services/search'
# ---------------------------

# Query the resource manager catalog to discover the application package
# ----------------------------------------------------------------------
application_package_id = "e563cc5596b02a189277de6228a9e2af"  # the one we just published ourselves
payload = {'uid': application_package_id,'format': 'json'}
r = requests.get(rm_endpoint, params=payload)
print(r.status_code, r.reason)

# Receive OpenSearch query response as json
response = r.json()

# The Application Package is extracted from the response 'offering'
application_package = yaml.safe_load(response['features'][0]['properties']['offering']['content']['#text'])

### Deploy the process - (POST /processes)

In [ ]:
# ADES endpoint
# -------------
endpoint = 'http://proc-ades.test.192.168.49.2.nip.io'
# -------------

# Prepare the request
token = "eyJhbGciOiJIUzI1NiIsImtpZCI6IlJTQTEifQ.eyJhY3RpdmUiOnRydWUsImV4cCI6MTU5MzUxNTU2NSwiaWF0IjoxNTkzNTExOTY1LCJuYmYiOm51bGwsInBlcm1pc3Npb25zIjpbeyJyZXNvdXJjZV9pZCI6ImI3Y2FkZTVjLTM3MmYtNGM4Ny1iZTgyLWE3OTU2NDk4ZTcyOSIsInJlc291cmNlX3Njb3BlcyI6WyJBdXRoZW50aWNhdGVkIiwib3BlbmlkIl0sImV4cCI6MTU5MzUxNTU2NCwicGFyYW1zIjpudWxsfV0sImNsaWVudF9pZCI6IjYxY2UyOGQ1LWFhMTYtNGRkYy04NDJmLWZjYzE1OGQzMTVmYSIsInN1YiI6bnVsbCwiYXVkIjoiNjFjZTI4ZDUtYWExNi00ZGRjLTg0MmYtZmNjMTU4ZDMxNWZhIiwiaXNzIjpudWxsLCJqdGkiOm51bGwsInBjdF9jbGFpbXMiOnsiYXVkIjpbIjYxY2UyOGQ1LWFhMTYtNGRkYy04NDJmLWZjYzE1OGQzMTVmYSJdLCJzdWIiOlsiZWIzMTQyMWUtMGEyZS00OTBmLWJiYWYtMDk3MWE0ZTliNzhhIl0sInVzZXJfbmFtZSI6WyJyaWNvbndheSJdLCJpc3MiOlsiaHR0cHM6Ly9lb2VwY2EtZGV2LmRlaW1vcy1zcGFjZS5jb20iXSwiZXhwIjpbIjE1OTM1MTU1NjQiXSwiaWF0IjpbIjE1OTM1MTE5NjQiXSwib3hPcGVuSURDb25uZWN0VmVyc2lvbiI6WyJvcGVuaWRjb25uZWN0LTEuMCJdfX0.d5qeaqLfl0oh9KigVrM_lT1hZMaOzQBFB7jjaKI3PjE"
deploy_headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json', 'Accept': 'application/json', 'Prefer': 'respond-async'}
# Request body defined by 'OGC API Processes'
deploy_payload = {'inputs': [{'id': 'applicationPackage',
                              'input': {'format': {'mimeType': 'application/xml'},
                                        'value': {'href': f'https://catalog.terradue.com/eoepca-services/search?uid={application_package_id}'}}}],
                  'outputs': [{'format': {'mimeType': 'string',
                                          'schema': 'string',
                                          'encoding': 'string'},
                               'id': 'deployResult',
                               'transmissionMode': 'value'}],
                  'mode': 'auto',
                  'response': 'raw'}

# OGC API Processes - deploy the application to the ADES
r = requests.post(f'{endpoint}/wps3/processes', json=deploy_payload, headers=deploy_headers)
print(r.status_code, r.reason)

### Check our application is deployed - (GET /processes)

In [ ]:
headers = {'Authorization': f'Bearer {token}','Content-Type': 'application/json', 'Accept': 'application/json'}
# OGC API Processes - list
# ------------------------
r = requests.get(f'{endpoint}/wps3/processes', headers=headers)
# ------------------------
print(r.status_code, r.reason)

# Interpret json response body
response = r.json()
for process in response['processes']:
    print('Process id: {}\n{} - {}\n'.format(process['id'], process['title'], process['abstract']))

### Check application details - (GET /processes/{id})
Need to understand application inputs in order to prepare our execute request

In [ ]:
process_id = 'vegetation_index_'
# OGC API Processes - details
# ---------------------------
r = requests.get(f'{endpoint}/wps3/processes/{process_id}', headers=headers)
# ---------------------------
print(r.status_code, r.reason)

# Interpret json response body
response = r.json()
inputs = response['process']['inputs']
print(json.dumps(inputs, indent=2))

### Execute - (POST /processes/{id}/jobs)

In [ ]:
# Prepare the request
process_id = 'vegetation_index_'
execution_headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json', 'Accept': 'application/json', 'Prefer': 'respond-async'}
# Request body defined by 'OGC API Processes'
# - input data by reference to catalogue record
# - area-of-interest defined by WKT POLYGON (Kangaroo Island)
execute_request = {'inputs': [{'id': 'input_reference',
                               'input': {'dataType': {'name': 'string'},
                                         'value': 'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808'}},
                              {'id': 'aoi',
                               'input': {'dataType': {'name': 'string'},
                                         'value': 'POLYGON((136.112726861895 -36.227897298303,137.333826362695 -36.2103069464338,137.305145407058 -35.2211228310596,136.099040812374 -35.2380875358202,136.112726861895 -36.227897298303))'}}],
                   'outputs': [{'format': {'mimeType': 'string',
                                           'schema': 'string',
                                           'encoding': 'string'},
                                'id': 'wf_output',
                                'transmissionMode': 'value'}],
                   'mode': 'async',
                   'response': 'raw'}

# OGC API Processes - submit execution request to the ADES
r = requests.post(f'{endpoint}/wps3/processes/{process_id}/jobs', json=execute_request, headers=execution_headers)
print(r.status_code, r.reason)
job_location = r.headers['Location']
print(f"Job status monitor = {job_location}")

### Monitor job status - (GET /processes/{id}/jobs/{job_id})

#### Quick check...

In [ ]:
r = requests.get(f'{endpoint}{job_location}', headers=headers)
print(r.status_code, r.reason)
response = r.json()
print(json.dumps(response, indent=2))

#### Poll status and wait for completion

In [ ]:
while response['status'] == 'running':
    r = requests.get(f'{endpoint}{job_location}', headers=headers)
    response = r.json()
    if response['status'] == 'failed': 
        print(response)
        break
    if response['status'] == 'successful':  
        print(response['links'][0]['href'])
        break
    else:
        print('Polling - {}'.format(response['status']))
        sleep(30)

### Processing Results - (GET /processes/{id}/jobs/{job_id}/result)

In [ ]:
# Request the results
# -------------------
r = requests.get(f'{endpoint}/{job_location}/result', headers=headers)
# -------------------
print(r.status_code, r.reason)

# Interpret the response
response = r.json()
results = json.loads(response['outputs'][0]['value']['inlineValue'])
print(json.dumps(results, indent=2))
stac_catalog_endpoint = results['stac:catalog']['href']
print(f"STAC catalogue results index = {stac_catalog_endpoint}")